In [14]:
from ROOT import TFile
from root_numpy import root2array, root2rec, tree2array
import array
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras import losses, optimizers

In [2]:
inpath = "/data_CMS/cms/tsculac/CJLST_NTuples/"
filename = "/ZZ4lAnalysis.root"
H1_files = ["VBFH125"]
H0_files = ["ggH125"]

In [3]:
def generate_training_data(H1_file, H0_file, branches, chunksize = 10):
    H1_curpos = 0
    H0_curpos = 0
    
    while H1_curpos < 20:
        # prepare training data chunk set by drawing events randomly from the two files
        H1_data = pd.DataFrame(root2array(inpath + H0_file + filename, treename = "ZZTree/candTree", 
                    branches = branches, start = H1_curpos, stop = H1_curpos + chunksize))

        H0_data = pd.DataFrame(root2array(inpath + H1_file + filename, treename = "ZZTree/candTree",
                    branches = branches, start = H0_curpos, stop = H0_curpos + chunksize))
                        
        H1_curpos += chunksize
        H0_curpos += chunksize

        # add the truth information
        H1_data["target"] = 1.0
        H0_data["target"] = 0.0

        data_chunk = pd.concat([H1_data, H0_data])

        yield data_chunk.sample(frac = 1)

In [4]:
branches = ["PFMET", "nCleanedJetsPt30"]

In [27]:
generator = generate_training_data(H0_files[0], H1_files[0], branches, chunksize = 10)

In [7]:
# make a simple Keras model, just for Hello World purposes
model = Sequential()

In [10]:
model.add(Dense(32, input_shape=(2,)))
model.add(Dense(32))
model.add(Dense(1))
model.add(Activation("softmax"))

In [16]:
sgd = optimizers.SGD(lr = 0.01)
model.compile(loss = "mean_squared_error", optimizer = sgd, metrics = ["accuracy"])

In [28]:
for training_data in generator:
    input_data = training_data[branches].as_matrix()
    target_data = training_data["target"].as_matrix()
    model.fit(input_data, target_data, epochs = 50, batch_size = 128)

Epoch 1/50
20/20 [==============================] - 0s - loss: 0.5000 - acc: 0.5000
Epoch 2/50
20/20 [==============================] - 0s - loss: 0.5000 - acc: 0.5000
Epoch 3/50
20/20 [==============================] - 0s - loss: 0.5000 - acc: 0.5000
Epoch 4/50
20/20 [==============================] - 0s - loss: 0.5000 - acc: 0.5000
Epoch 5/50
20/20 [==============================] - 0s - loss: 0.5000 - acc: 0.5000
Epoch 6/50
20/20 [==============================] - 0s - loss: 0.5000 - acc: 0.5000
Epoch 7/50
20/20 [==============================] - 0s - loss: 0.5000 - acc: 0.5000
Epoch 8/50
20/20 [==============================] - 0s - loss: 0.5000 - acc: 0.5000
Epoch 9/50
20/20 [==============================] - 0s - loss: 0.5000 - acc: 0.5000
Epoch 10/50
20/20 [==============================] - 0s - loss: 0.5000 - acc: 0.5000
Epoch 11/50
20/20 [==============================] - 0s - loss: 0.5000 - acc: 0.5000
Epoch 12/50
20/20 [==============================] - 0s - loss: 0.5000 - a